In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# 1. 메모리에 임시 저장해서 같은 질문을 빠르게 답변
# DP 같은 느낌
# 컴퓨터가 꺼지면 다시 실행해야 됨(휘발성)
set_llm_cache(InMemoryCache())
set_debug(True)  # json 형식으로 debug를 출력해서 확인 가능

# 2. DB를 생성해서 db에 저장
set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

Arrr matey! Me favorite grub be a hearty plate of salted fish and hardtack! Aye, nothing beats the taste of the sea on me tongue! Arrr!

AIMessageChunk(content='Arrr matey! Me favorite grub be a hearty plate of salted fish and hardtack! Aye, nothing beats the taste of the sea on me tongue! Arrr!')